In [ ]:
## Notebook env: sequencing_trblsht_py (sequencing_trblsht_py kernel)
## this notebook troubleshoots the low guide count in the cellranger output of the repeated CRISPRa screen

In [1]:
## possibilities to work through:
# contamination of sequences from other guide libraries of Brian
# the library actually transfected is different than what Brian thought and the list of guides he gave me
# I messed up in cell ranger guide library input (not true, Brian confirmed)


### plan is to load in the feature barcode reads, pull out the sequences at the expected barcode location and count them
### next plan is to match library sequences (botht the given library and other libraries Brian uses) to feature barcode reads
### in the end need to explain why only about ~1.5% of the reads are matching the inputed library


In [1]:
import dnaio
import pandas as pd
import os
import multiprocessing as mp
import re
import regex

In [44]:
## load in CRISPRa sequences and fastq files containing feature barcodes
guide_barcodes = pd.read_csv('/home/ssobti/projects/heterogeneity_brian/data/072623_MD231_CRISPRa_repeat_screen/cellranger/feature_ref_arg/GEM1_GEM2_CRISPRa_features.csv')
guide_barcodes = guide_barcodes['sequence'].values

path = '/home/ssobti/projects/heterogeneity_brian/data/072623_MD231_CRISPRa_repeat_screen/BWHG37/'
sample_names = ['SL424_S3_L004_R2_001.fastq.gz', 'SL425_S4_L004_R2_001.fastq.gz']

In [ ]:
### run a multiprocess to obtain the reads
os.system("taskset -p --cpu-list 0-19 %d" % os.getpid())

import multiprocessing as mp

def count_reads1(sample, barcode):
    count = 0
    compiled_barcode = regex.compile('(%s){e<=1}' % barcode)
    for idx, string in enumerate(sample):
        if regex.findall(compiled_barcode, string):
            count += 1
    print(f'{barcode} in SL424_ROI_GEM1 has {count} reads')
    return count

def count_reads2(sample, barcode):
    count = 0
    compiled_barcode = regex.compile('(%s){e<=1}' % barcode)
    for idx, string in enumerate(sample):
        if regex.match(compiled_barcode, string):
            count += 1
    print(f'{barcode} in SL425_ROI_GEM2 has {count} reads')
    return count

def obtain_reads(path, sample_name, start = None, end = None):
    sample = dnaio.open(path + sample_name)
    out = []
    if start == None and end == None:
        for record in sample:
            out.append(str(record.sequence))

    else:
        for record in sample:
            out.append(str(record[start:end].sequence))
    
    return out


pid 3093259's current affinity list: 0-55
pid 3093259's new affinity list: 0-19


In [47]:
SL424_ROI_GEM1 = obtain_reads(path, sample_names[0], 28, 55)
SL425_ROI_GEM2 = obtain_reads(path, sample_names[1], 28, 55)


if __name__ == '__main__':
    p1 = mp.Pool(10)
    p2 = mp.Pool(10)
    SL424_ROI_GEM1_counts = p1.starmap_async(count_reads1, [(SL424_ROI_GEM1, barcode) for barcode in guide_barcodes])
    p1.close()
    p1.join()
    SL425_ROI_GEM2_counts = p2.starmap_async(count_reads2, [(SL425_ROI_GEM2, barcode) for barcode in guide_barcodes])
    p2.close()
    p2.join()

KeyboardInterrupt: 

GGGCGCGCGCCCAGACTCCT in SL424_ROI_GEM1 has 258734 reads


In [ ]:
SL424_ROI_GEM1_counts_final = [r.get() for r in SL424_ROI_GEM1_counts]
SL425_ROI_GEM2_counts_final = [r.get() for r in SL425_ROI_GEM2_counts]

In [ ]:
#### SAVING 

import pickle
import shelve
filenm = '/home/ssobti/projects/heterogeneity_brian/output_data/CRISPRa_repeat_screen/read_trblsht_save.out'
my_shelf = shelve.open(filenm,'n') # 'n' for new
vec = ['SL424_ROI_GEM1_counts_final', 'SL425_ROI_GEM2_counts_final', 'SL424_ROI_GEM1', 'SL425_ROI_GEM2'] ## put objects you want to save in quotes separated by commas
for key in vec:
    try:
        my_shelf[key] = globals()[key]
    except TypeError:
        #
        # __builtins__, my_shelf, and imported modules can not be shelved.
        #
        print('ERROR shelving: {0}'.format(key))
    except pickle.PicklingError:
        print('ERROR shelving: {0}'.format(key))
    except KeyError:
        print('ERROR shelving: {0}'.format(key))
my_shelf.close()



In [1]:
## ideal range for kmer is position 31-51
## pull out kmers starting from 28-34 and ending from 48-54 making sure each kmer has length 20
## do fuzzy matching to among the kmers to count them




